# Importing Dependencies

In [9]:
import pandas as pd
import numpy as np
import xlsxwriter
import math
import pandas_datareader.data as web
import schwabdev
from scipy import stats
from datetime import datetime
from dateutil.relativedelta import relativedelta
%matplotlib inline

# Momentum Investing
- the strategy in investing in the stocks that increased in price the most

# Collecting S&P 500 constituents

In [2]:
sp_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_constituents = pd.read_html(sp_url, header=0)[0]

In [3]:
sp500_constituents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 503 non-null    object
 1   Security               503 non-null    object
 2   GICS Sector            503 non-null    object
 3   GICS Sub-Industry      503 non-null    object
 4   Headquarters Location  503 non-null    object
 5   Date added             503 non-null    object
 6   CIK                    503 non-null    int64 
 7   Founded                503 non-null    object
dtypes: int64(1), object(7)
memory usage: 31.6+ KB


In [5]:
sp500_constituents = sp500_constituents['Symbol'].to_list()

### changing format to match Schwab's

In [6]:
sp500_constituents = [x.replace('.', '/') for x in sp500_constituents]

# Setting up Schwab API

In [4]:
from key import api_key, api_secret

In [48]:
client = schwabdev.Client(app_key=api_key, app_secret=api_secret, capture_callback=False)

# Creating Datafram
- Basing this strategy in a 6 month change

In [22]:
my_columns = ['Ticker', 'Price (6 months)', 'Price (now)', 'Net Change', 'Number of Shares to Buy']

In [10]:
today = datetime.today()
six_months = today - relativedelta(months = 6)

In [49]:
entries = []
for symbol in sp500_constituents:
    price_history = client.price_history(symbol, periodType='month', period = 6, startDate=six_months, endDate=today).json()
    price_now = price_history['candles'][-1]['close']
    price_6_months = price_history['candles'][0]['close']
    net_change = round(((price_now - price_6_months)/ price_6_months) * 100, 2)
    entries.append(
        [
            symbol,
            price_6_months,
            price_now,
            net_change,
            'N/a'
        ]
    )

In [50]:
momentum_df = pd.DataFrame(entries, columns=my_columns)

### Sorting by biggest increases

In [51]:
momentum_df.sort_values(by='Net Change', ascending=False, inplace=True)

In [52]:
momentum_df.head()

,Ticker,Price (6 months),Price (now),Net Change,Number of Shares to Buy
360,PLTR,44.86,92.770,106.80,N/a
171,EQT,37.48,51.890,38.45,N/a
468,VRSN,181.26,249.185,37.47,N/a
308,MCK,507.41,695.040,36.98,N/a
365,PAYC,164.63,217.980,32.41,N/a


# Creating Excel Output with all data

In [53]:
momentum_df_copy = momentum_df.copy()

In [54]:
momentum_df_copy = momentum_df_copy.drop('Number of Shares to Buy', axis=1)
momentum_df_copy.head()

,Ticker,Price (6 months),Price (now),Net Change
360,PLTR,44.86,92.770,106.80
171,EQT,37.48,51.890,38.45
468,VRSN,181.26,249.185,37.47
308,MCK,507.41,695.040,36.98
365,PAYC,164.63,217.980,32.41


In [150]:
writer = pd.ExcelWriter('total_data.xlsx', engine='xlsxwriter')

momentum_df_copy.to_excel(writer, sheet_name='Movers', index=False)

### Formatting

In [151]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [152]:
dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [153]:
percent_format = writer.book.add_format(
    {
        'num_format': '0.0%',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

### Applying Formats

In [154]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price (6 months)', dollar_format],
    'C': ['Price (now)', dollar_format],
    'D': ['Net Change', percent_format]
}

In [155]:
for column in column_formats.keys():
    writer.sheets['Movers'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Movers'].write(f'{column}1', column_formats[column][0], column_formats[column][1])
    

In [156]:
writer.close()

# Keeping Top 50 Movers

In [138]:
final_df = momentum_df.copy()

In [139]:
final_df = final_df.head(50)
final_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 360 to 488
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Ticker                   50 non-null     object 
 1   Price (6 months)         50 non-null     float64
 2   Price (now)              50 non-null     float64
 3   Net Change               50 non-null     float64
 4   Number of Shares to Buy  50 non-null     object 
dtypes: float64(3), object(2)
memory usage: 2.3+ KB


In [140]:
final_df.reset_index(inplace=True)
final_df.drop('index', axis = 1, inplace=True)
final_df

,Ticker,Price (6 months),Price (now),Net Change,Number of Shares to Buy
0,PLTR,44.86,92.7700,106.80,N/a
1,EQT,37.48,51.8900,38.45,N/a
2,VRSN,181.26,249.1850,37.47,N/a
3,MCK,507.41,695.0400,36.98,N/a
4,PAYC,164.63,217.9800,32.41,N/a
5,TTWO,161.79,213.9100,32.21,N/a
6,TKO,115.25,150.5200,30.60,N/a
7,AXON,444.52,577.8400,29.99,N/a
8,NFLX,754.68,975.1212,29.21,N/a
9,CRWD,300.62,388.2000,29.13,N/a


# Calculating the Number of Shares to Buy

In [141]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of the porfolio: ')

    try:
        float(portfolio_size)
    except ValueError:
        print('Invalid input! \n Please try again.')
        portfolio_size = input('Enter the size of the porfolio: ')


In [142]:
portfolio_input()

In [143]:
print(portfolio_size)

100000


In [144]:
position_size = float(portfolio_size) / len(final_df.index)
position_size

2000.0

In [148]:
for i in range(0, len(final_df)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_df.loc[i, 'Price (now)'])

In [149]:
final_df.head()

,Ticker,Price (6 months),Price (now),Net Change,Number of Shares to Buy
0,PLTR,44.86,92.770,106.80,21
1,EQT,37.48,51.890,38.45,38
2,VRSN,181.26,249.185,37.47,8
3,MCK,507.41,695.040,36.98,2
4,PAYC,164.63,217.980,32.41,9


# Exporting to Excel

In [161]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')

final_df.to_excel(writer, sheet_name='Recommended Trades', index=False)

In [162]:
integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

In [163]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Price (6 months)', dollar_format],
    'C': ['Price (now)', dollar_format],
    'D': ['Net Change', percent_format],
    'E': ['Number of Shares to Buy', integer_format]
}

In [164]:
for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [165]:
writer.close()